## Learn Python Meta Programming

In [74]:
## Typing of any variable
class Meta1:
    pass

def func1():
    pass

a = 1
b = 1.2
c = "hello"
d = True
e = None
f = Meta1()

print(a, b, c, d, e, f, func1)
print(type(a), type(b), type(c), type(d), type(e), type(f), type(func1))

# So type() function return a typing class of variable (even int, float, str and bool in Python is a class)

1 1.2 hello True None <__main__.Meta1 object at 0x7fd150432af0> <function func1 at 0x7fd15037f670>
<class 'int'> <class 'float'> <class 'str'> <class 'bool'> <class 'NoneType'> <class '__main__.Meta1'> <class 'function'>


In [75]:
import inspect

print(inspect.isclass(a))
print(inspect.isclass(type(a)))
print(inspect.isclass(f))
print(inspect.isclass(type(f)))
print(inspect.isclass(inspect.isclass(type(f))))
print(inspect.isclass(type(inspect.isclass(type(f)))))
print(type(type(f)))

# Looks like everything in Python is a object => best place for meta programming :)

False
True
False
True
False
True
<class 'type'>


In [76]:
# Some common namespaces of a class

# So let print more readable with this command
from pprint import pprint

class Meta2:
    ''' Okay, this is document about Meta2 class '''
    var2 = 42
    def __init__(self) -> None:
        print("Hello from Meta2")
    
    def echo(self):
        ''' this is echoooooo function '''
        print("Echooooo")
        print(self)

print(type(Meta2.__dict__)) 
pprint(Meta2.__dict__) # Namespace of class

echo_from_meta2 = Meta2.__dict__.get('echo') # Try to get echo func from class namespace
try:
    echo_from_meta2("KhanhIceTea") # Pass anything to param "self", works ! self is not variable for current object in class context
    echo_from_meta2() # Keep param "self" empty => exception 
except Exception as e:
    print(e)
pprint(echo_from_meta2.__dict__) # function doesn't have namespace like class

<class 'mappingproxy'>
mappingproxy({'__dict__': <attribute '__dict__' of 'Meta2' objects>,
              '__doc__': ' Okay, this is document about Meta2 class ',
              '__init__': <function Meta2.__init__ at 0x7fd1410df0d0>,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'Meta2' objects>,
              'echo': <function Meta2.echo at 0x7fd1410df1f0>,
              'var2': 42})
Echooooo
KhanhIceTea
echo() missing 1 required positional argument: 'self'
{}


In [77]:
## Ok, deep dive into type() function

# print type func signature
print(type.__doc__)

# interesting ! so let try call type with 3 args as docs say "type(name, bases, dict) -> a new type"
# Becareful, bases arg is tupple of base classes
CloneMeta3 = type("CloneMeta2", (Meta2,), {"var3": 4})

# IDE with warning this class is not defined, but is it
print(CloneMeta3) # new class is belongs to current module run the type() func
print(CloneMeta3.var2, CloneMeta3.var3) # new SubClass inherits base classes
clone1 = CloneMeta3()
clone1.echo()
print(type(CloneMeta3))


type(object_or_name, bases, dict)
type(object) -> the object's type
type(name, bases, dict) -> a new type
<class '__main__.CloneMeta2'>
42 4
Hello from Meta2
Echooooo
<class 'type'>


In [78]:
# OK, nice lets try convert this
# class A:
#     var1 = 1
# class B(A):
#     var2 = 2
# by using type() function

A = type("A", (), {"var1": 1})
B = type("B", (A,), {"var2": 2})

print(A)
print(A.var1)
print(B)
print(B.var1)
print(B.var2)

<class '__main__.A'>
1
<class '__main__.B'>
1
2


## Decorators

In [79]:
def echo1(msg: str):
    ''' This func will echo loud to input message '''
    print(f"echo1 is echoing the msg '{msg}'")

print(echo1)
echo1("Hello world!")

# Now we create a decorator function to wrap (like Proxy Pattern in OOP)
def decor1(func):
    return func

decor1_echo1 = decor1(echo1) # this simplest decorator function
decor1_echo1("He he ha ha") # The same result as echo1 func

print(decor1_echo1)
print(decor1_echo1 == echo1)
print("It's have the same pointer address b/c return same function")



<function echo1 at 0x7fd141139040>
echo1 is echoing the msg 'Hello world!'
echo1 is echoing the msg 'He he ha ha'
<function echo1 at 0x7fd141139040>
True
It's have the same pointer address b/c return same function


In [80]:
# Go to more complex decorator

def decor2(func):
    def wrap(*args, **kwargs):
        print("Ehhhhh hemmmmm, calling from decor2 wrap function")
        ret = func(*args, **kwargs) # store returned value
        print("Yay, finished !")
        return ret
    return wrap

@decor2
def echo2(msg: str):
    ''' This func will echo loud to input message '''
    print(f"echo2 is echoing the msg '{msg}'")

decor2_echo1 = decor2(echo1)

print("=== Try to deepdive into some common namespaces of decorated function decor1_echo1")
pprint(decor2_echo1.__name__)
pprint(decor2_echo1.__dict__)
pprint(decor2_echo1.__dir__)
pprint(decor2_echo1.__doc__)
pprint(decor2_echo1.__annotations__)
pprint(decor2_echo1.__call__)

print("=== Try to deepdive into some common namespaces of decorated function echo2")
pprint(echo2.__name__)
pprint(echo2.__dict__)
pprint(echo2.__dir__)
pprint(echo2.__doc__)
pprint(echo2.__annotations__)
pprint(echo2.__call__)
print("=> As you can see, all namespaces of echo1, echo2 wouldn't be copied to final functions")


=== Try to deepdive into some common namespaces of decorated function decor1_echo1
'wrap'
{}
<built-in method __dir__ of function object at 0x7fd141119280>
None
{}
<method-wrapper '__call__' of function object at 0x7fd141119280>
=== Try to deepdive into some common namespaces of decorated function echo2
'wrap'
{}
<built-in method __dir__ of function object at 0x7fd141139f70>
None
{}
<method-wrapper '__call__' of function object at 0x7fd141139f70>
=> As you can see, all namespaces of echo1, echo2 wouldn't be copied to final functions


In [81]:
# So we have to copy namespace we need in decorated function
def decor3(func):
    def wrap(*args, **kwargs):
        print("Ehhhhh hemmmmm, calling from decor3 wrap function")
        ret = func(*args, **kwargs) # store returned value
        print("Yay, finished !")
        return ret
    wrap.__name__ = func.__name__
    wrap.__doc__ = func.__doc__
    wrap.__annotations__ = func.__annotations__
    return wrap

@decor3
def echo3(msg: str):
    ''' This func will echo loud to input message '''
    print(f"echo3 is echoing the msg '{msg}'")

echo3("OK")
print("=== Try to deepdive into some common namespaces of decorated function echo3")
pprint(echo3.__name__)
pprint(echo3.__doc__)
pprint(echo3.__annotations__)
print("=== See ! echo3 has its original namespaces")

Ehhhhh hemmmmm, calling from decor3 wrap function
echo3 is echoing the msg 'OK'
Yay, finished !
=== Try to deepdive into some common namespaces of decorated function echo3
'echo3'
' This func will echo loud to input message '
{'msg': <class 'str'>}
=== See ! echo3 has its original namespaces


In [82]:
# Copy namespace by using built-in function called wrap in functools module
from functools import wraps

def decor4(func):
    ''' 
    all it do is copy func's namespaces to wrap's namespaces :
        (__name__, __qualname__, __module__, __doc__, __annotations__)
    & updates __dict__ elements
    & point wrap.__wrapped__ to original func
    '''
    @wraps(func) 
    def wrap(*args, **kwargs):
        print("Ehhhhh hemmmmm, calling from decor4 wrap function")
        ret = func(*args, **kwargs) # store returned value
        print("Yay, finished !")
        return ret
    return wrap

@decor4
def echo4(msg: str):
    ''' This func will echo loud to input message '''
    print(f"echo4 is echoing the msg '{msg}'")

echo3("OK")
print("=== Try to deepdive into some common namespaces of decorated function echo4")

pprint(echo4.__name__)
pprint(echo4.__qualname__)
pprint(echo4.__module__)
pprint(echo4.__doc__)
pprint(echo4.__annotations__)
pprint(echo4.__wrapped__) # original echo4
pprint(echo4) # new echo4
print("=== See ! echo3 has its original namespaces")

Ehhhhh hemmmmm, calling from decor3 wrap function
echo3 is echoing the msg 'OK'
Yay, finished !
=== Try to deepdive into some common namespaces of decorated function echo4
'echo4'
'echo4'
'__main__'
' This func will echo loud to input message '
{'msg': <class 'str'>}
<function echo4 at 0x7fd141119ee0>
<function echo4 at 0x7fd1411190d0>
=== See ! echo3 has its original namespaces


In [83]:
# Okay, you can see `wraps` built-in function is used as decorator it self :lol: and it has additional parameters,
# let's try !
# Basically, it's just a function that return a decorator

def decor5(bye_msg):
    def inner_decor(func):
        @wraps(func) 
        def wrap(*args, **kwargs):
            print("Ehhhhh hemmmmm, calling from decor5 wrap function")
            ret = func(*args, **kwargs) # store returned value
            print("Yay, finished ! ", bye_msg) # use `bye_msg` parameter here (like a closure)
            return ret
        return wrap
    return inner_decor

@decor5("Bye bye buddy !")
def echo5(msg: str):
    ''' This func will echo loud to input message '''
    print(f"echo5 is echoing the msg '{msg}'")

echo5("Hi Python community !")

Ehhhhh hemmmmm, calling from decor5 wrap function
echo5 is echoing the msg 'Hi Python community !'
Yay, finished !  Bye bye buddy !


In [84]:
# Let try decorator on classes

def decor6(cls): # cls is decorated Class
    for name, val in vars(cls).items():
        print(name, "=>", val)
    return cls

@decor6
class Meta6:
    ''' This is docs of Meta6 '''
    var1 = 1
    def __init__(self, a) -> None:
        self.a = a

    @decor5("Bye universe ! Snapppp !") # use decorator in class method
    def hi(self, msg):
        print(msg, self.a)

m6 = Meta6(42)
m6.hi("Number of universe is")


__module__ => __main__
__doc__ =>  This is docs of Meta6 
var1 => 1
__init__ => <function Meta6.__init__ at 0x7fd153528700>
hi => <function Meta6.hi at 0x7fd141119c10>
__dict__ => <attribute '__dict__' of 'Meta6' objects>
__weakref__ => <attribute '__weakref__' of 'Meta6' objects>
Ehhhhh hemmmmm, calling from decor5 wrap function
Number of universe is 42
Yay, finished !  Bye universe ! Snapppp !


## Metaclasses

In [85]:
## What if, what ifff Meta6 has needed multiple decorators and has multi subclasses, we have to put decorators in each of them (subclasses)
## OR
## Use Metaclasses

# We want to add logging in every method within its name starts 'post'
import time

def stopwatch(func):
    def wrap(*args, **kwargs):
        start = time.time()
        print("[ Start stopwatch ]")
        ret = func(*args, **kwargs)
        time_lapsed = (time.time() - start) * 1E6
        print(f"[ End stopwatch => above function take {time_lapsed:.2f}μs to finish ]")
        return ret
    return wrap

class LoggingMetaClass(type):
    def __new__(cls, name, bases, dct : dict):
        print("class => ", cls)
        print("name => ", name)
        print("bases => ", bases)
        print("dct (__dict__) => ", dct)

        # We inject stopwatch decorator here, remember we inject __dict__ before
        for key, val in dct.items():
            if callable(val) and str(key).startswith("post"):
                dct[key] = stopwatch(val)

        x = super().__new__(cls, name, bases, dct)
        return x

class Meta7(metaclass=LoggingMetaClass):
    def __init__(self, name):
        self.name = name

    def getFromHackerNews(self):
        print(self.name, "is getting from HN")

    def postToGoogle(self):
        print(self.name, "is posting to Google")
    
    def postToWikipedia(self):
        print(self.name, "is posting to Wikipedia")

m7 = Meta7("Khanh")
m7.getFromHackerNews()
m7.postToGoogle()
m7.postToWikipedia()


class =>  <class '__main__.LoggingMetaClass'>
name =>  Meta7
bases =>  ()
dct (__dict__) =>  {'__module__': '__main__', '__qualname__': 'Meta7', '__init__': <function Meta7.__init__ at 0x7fd1410dfb80>, 'getFromHackerNews': <function Meta7.getFromHackerNews at 0x7fd1410df040>, 'postToGoogle': <function Meta7.postToGoogle at 0x7fd1410ed940>, 'postToWikipedia': <function Meta7.postToWikipedia at 0x7fd1410ed670>}
Khanh is getting from HN
[ Start stopwatch ]
Khanh is posting to Google
[ End stopwatch => above function take 51.02μs to finish ]
[ Start stopwatch ]
Khanh is posting to Wikipedia
[ End stopwatch => above function take 54.12μs to finish ]


In [86]:
## Apply metaclass into Abstract base classes

from abc import ABCMeta, abstractmethod

class Animal(metaclass=ABCMeta):

    @abstractmethod
    def sound(self):
        pass

class Bird(Animal):
    pass

b = Bird() # Should be TypeError: Can't instantiate abstract class Bird with abstract methods sound

TypeError: Can't instantiate abstract class Bird with abstract methods sound

In [ ]:
class Dog(Animal):
    def sound(self):
        print("Woof ! Woof !")

class Fox(Animal):
    def sound(self):
        print("What does the fox sayyyyy !?!")

d = Dog()
f = Fox()
d.sound()
f.sound()

Woof ! Woof !
What does the fox sayyyyy !?!


In [99]:
# More use cases, metaclasses is not only way to achieve (we can even use decorator like above), but if things go complicated, try Metaclasses
# Real usecases :
# - Abstract Handlers
# - Database Models
# - Worker
# ...
import queue

class JobQueue:
    tasks = {}
    
    def __init__(self):
        # Simple FIFO queue
        self.queues = queue.Queue()

    def queue(self, task, payload):
        self.queues.put((task, payload))
    
    def work(self):
        while not self.queues.empty():
            (task, payload) = self.queues.get()
            if task in JobQueue.tasks:
                cls_hanlder = JobQueue.tasks[task]
                cls_hanlder.handle(payload)
            else:
                print(f"No handler for task {task}")

class Worker(type):
    def __new__(cls, name, bases, dct : dict):
        ''' Register to handle task name right on class declaration '''
        x = super().__new__(cls, name, bases, dct)
        JobQueue.tasks[x.task_name] = x
        return x

class SendEmailWorker(metaclass=Worker):
    task_name = 'send_email'

    def handle(job):
        print("sending email", job)

class SendSmsWorker(metaclass=Worker):
    task_name = 'send_sms'

    def handle(job):
        print("sending sms", job)

print(JobQueue.tasks) # See, we got job handlers
jq = JobQueue()
jq.queue('send_email', 'a@b.com')
jq.queue('send_sms', '0987654321')
jq.queue('call', '0987654321')
jq.queue('send_email', 'a@b.com')
jq.work()

{'send_email': <class '__main__.SendEmailWorker'>, 'send_sms': <class '__main__.SendSmsWorker'>}
sending email a@b.com
sending sms 0987654321
No handler for task call
sending email a@b.com
